sgdclassifier | bag of sites | 0.83341

In [1]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import os
import pickle
import numpy as np
import pandas as pd
# from scipy.sparse import csr_matrix
from sklearn.model_selection import cross_val_score, TimeSeriesSplit
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
# !unzip capstone_user_identification.zip

### Обработка входных данных

In [2]:
PATH_TO_DATA = '../capstone_user_identification'
# PATH_TO_DATA = '../input/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2'

train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'),
                       index_col='session_id')
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'),
                      index_col='session_id')
train_df.shape, test_df.shape

((253561, 21), (82797, 20))

In [3]:
train_df.sort_values(by='time1', inplace=True)

In [4]:
train_test_df = pd.concat([train_df, test_df])
train_test_df_sites = train_test_df[['site%d' % i for i in range(1, 11)]].fillna(0).astype('int')

In [5]:
# !mkdir tmp

In [5]:
train_test_df_sites[:253561].to_csv('tmp/train.csv', header=False, index=False, sep=' ')
train_test_df_sites[253561:].to_csv('tmp/test.csv', header=False, index=False, sep=' ')

In [6]:
%%time
vectorizer = CountVectorizer(ngram_range=(1, 2), max_features=20000)
with open('tmp/train.csv') as train:
    X_train_sparse = vectorizer.fit_transform(train)
with open('tmp/test.csv') as test:
    X_test_sparse = vectorizer.transform(test)
y = train_df['target']
print(X_train_sparse.shape)

(253561, 20000)
CPU times: user 5.74 s, sys: 62.2 ms, total: 5.81 s
Wall time: 6.21 s


### Обучение модели

In [7]:
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1)

In [8]:
%%time
time_split = TimeSeriesSplit(n_splits=10)

cv_scores = cross_val_score(sgd_logit, X_train_sparse, y, cv=time_split, scoring='roc_auc')

CPU times: user 17.4 s, sys: 310 ms, total: 17.7 s
Wall time: 3.52 s


In [9]:
cv_scores, cv_scores.mean()

(array([0.78413451, 0.60730791, 0.8783136 , 0.96134666, 0.84649539,
        0.89894634, 0.93064162, 0.85145182, 0.93189001, 0.88853111]),
 0.8579058965518641)

0.85791

### Предсказание на тестовой выборке

In [94]:
sgd_logit.fit(X_train_sparse, y)
logit_test_pred_proba = sgd_logit.predict_proba(X_test_sparse)

In [95]:
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [96]:
write_to_submission_file(logit_test_pred_proba[:, 1], 'result.csv')

Бейзлайн sgd_logit_benchmark: 0.91273 побит